# Seminar 1. CV. Simple CNN

In [ ]:
from IPython import display
import numpy as np
import random
from tqdm import tqdm
import torch

In [ ]:
# fix all seeds
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## Пайплайн обучения CNN

In [ ]:
from copy import deepcopy

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns

!pip install mnist
import mnist

import torch.nn as nn
import torch.nn.functional as F

sns.set()

In [ ]:
images = mnist.train_images() / 255
labels = mnist.train_labels()

X_train, X_valid, y_train, y_valid = train_test_split(images, labels)

### Dataloader

In [ ]:
class MNISTDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __getitem__(self, item):
        pass
    
    def __len__(self):
        pass

In [ ]:
train_dataset = MNISTDataset(X_train, y_train)
valid_dataset = MNISTDataset(X_valid, y_valid)

In [ ]:
batch_size = 32
n_workers = 0

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                                        shuffle=True, num_workers=n_workers)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                                      shuffle=False, num_workers=n_workers)

### Check data

In [ ]:
img, label = next(iter(train_loader))
print(img.shape, label.shape)

### Simple CNN

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        pass 

    def forward(self, x):
        pass

In [ ]:
block = BasicConv2d(in_channels=32, out_channels=64, kernel_size=(3, 3))

x = torch.ones(1, 32, 28, 28)
out = block(x)
print(out.shape)

In [ ]:
class SimpleLayer(nn.Module):
    def __init__(self, in_channels, out_channels, conv=BasicConv2d, n=1, *args, **kwargs):
        super(SimpleLayer, self).__init__()
        self.block = nn.Sequential(
            conv(in_channels, out_channels, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.Dropout2d(0.2),
            nn.MaxPool2d((2, 2))
        )
    
    def forward(self, x):
        x = self.block(x)
        return x

In [ ]:
block = SimpleLayer(in_channels=32, out_channels=64, kernel_size=(3, 3))
print(block)

x = torch.ones(1, 32, 28, 28)
out = block(x)
print(out.shape)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, in_channels=3, n_classes=5, blocks_sizes=(32, 64, 128), *args, **kwargs):
        super(SimpleCNN, self).__init__()
        
        self.fc_dim = 128
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([
            SimpleLayer(in_channels, blocks_sizes[0], kernel_size=3, *args, **kwargs),
            *[SimpleLayer(in_channels, out_channels, kernel_size=3, *args, **kwargs)
              for in_channels, out_channels in self.in_out_block_sizes]
        ])
        
        self.fc = nn.Linear(self.fc_dim, n_classes)
    
    def forward(self, x):
        
        for block in self.blocks:
            x = block(x)
        
        bs, dim, h, w = x.shape
        x = x.view(bs, -1)
        
        out = self.fc(x)
        return out

In [ ]:
model = SimpleCNN(in_channels=1, n_classes=10)
print(model)

x = torch.ones(32, 1, 28, 28)
out = model(x)
print('Output shape:', out.shape)

### Init model and criterion

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

### Train loop

In [ ]:
def train(model, optimizer, n_epoch=20, batch_size=256, device="cpu"):
    train_logs = {"Train Loss": [0,], "Steps": [0,]}
    valid_logs = {"Valid Loss": [0,], "Valid Accuracy": [0,], "Steps": [0,]}
    step = 0
    best_valid_loss = np.inf
    best_model = None

    for i in range(n_epoch):
        for x_batch, y_batch in tqdm(train_loader, desc=f'train {i}/{n_epoch}'):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                # pass code
                loss = None
                pass 
            
            step += 1
            train_logs["Train Loss"].append(loss.detach().item())
            train_logs["Steps"].append(step)

        sum_loss = 0
        sum_acc = 0
        count_valid_steps = 0
        with torch.no_grad():
            for x_batch, y_batch in tqdm(val_loader, desc=f'val {i}/{n_epoch}'):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                sum_loss += loss.item()
                sum_acc += accuracy_score(y_batch.cpu().numpy(), np.argmax(predictions.cpu().numpy(), axis=1))
                count_valid_steps += 1

            valid_logs["Valid Loss"].append(sum_loss / count_valid_steps)
            valid_logs["Valid Accuracy"].append(sum_acc / count_valid_steps)
            valid_logs["Steps"].append(step)

            if best_valid_loss > sum_loss / count_valid_steps:
                best_valid_loss = sum_loss / count_valid_steps
                best_model = deepcopy(model)

    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    sns.lineplot(x="Steps", y="Train Loss", data=train_logs, ax=ax[0])
    sns.lineplot(x="Steps", y="Valid Loss", data=valid_logs, ax=ax[1])
    sns.lineplot(x="Steps", y="Valid Accuracy", data=valid_logs, ax=ax[2])
    plt.plot()

    return best_model, train_logs, valid_logs

In [ ]:
net, _, _ = train(model, optimizer, 1)

### Logging

Logging systems:
- [Tensorboard](https://pytorch.org/docs/stable/tensorboard.html)
- [WandB](https://www.wandb.com/)

In [ ]:
%load_ext tensorboard

In [ ]:
from datetime import datetime
from pathlib import Path

from torch.utils.tensorboard import SummaryWriter

In [ ]:
def train(model, optimizer, n_epoch=20, batch_size=256, device="cpu"):
    writer = SummaryWriter(Path("logs") / datetime.now().strftime("%Y%m%d-%H%M%S"))
    step = 0
    best_valid_loss = np.inf
    best_model = None

    model.to(device)
    for i in range(n_epoch):
        for x_batch, y_batch in tqdm(train_loader, desc=f'train {i}/{n_epoch}'):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                # pass code
                pass
            
            step += 1
            writer.add_scalar("Train Loss", loss.detach().item(), step)

        sum_loss = 0
        sum_acc = 0
        count_valid_steps = 0
        with torch.no_grad():
            for x_batch, y_batch in tqdm(val_loader, desc=f'val {i}/{n_epoch}'):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(x_batch)
                loss = criterion(predictions, y_batch)
                sum_loss += loss.item()
                sum_acc += accuracy_score(y_batch.cpu().numpy(), np.argmax(predictions.cpu().numpy(), axis=1))
                count_valid_steps += 1

            writer.add_scalar("Valid Loss", sum_loss / count_valid_steps, step)
            writer.add_scalar("Valid Accuracy", sum_acc / count_valid_steps, step)

            if best_valid_loss > sum_loss / count_valid_steps:
                best_valid_loss = sum_loss / count_valid_steps
                best_model = deepcopy(model)

    return best_model

In [ ]:
net = train(model, optimizer, n_epoch=1, device="cuda:0")

In [ ]:
%tensorboard --logdir logs

## Spoiler - train loop with [Catalyst](https://github.com/catalyst-team/catalyst)

- [A comprehensive step-by-step guide to basic and advanced features](https://github.com/catalyst-team/catalyst#step-by-step-guide)
- [Docs](https://catalyst-team.github.io/catalyst/)
- [What is Runner?](https://catalyst-team.github.io/catalyst/api/core.html#runner)

In [ ]:
! pip install catalyst
import catalyst
catalyst.__version__

In [ ]:
import os
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl, utils
from catalyst.data.transforms import ToTensor
from catalyst.contrib.datasets import MNIST

model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

loaders = {
    "train": DataLoader(
        MNIST(os.getcwd(), train=True, download=True, transform=ToTensor()), batch_size=32
    ),
    "valid": DataLoader(
        MNIST(os.getcwd(), train=False, download=True, transform=ToTensor()), batch_size=32
    ),
}

runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)
# model training
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=1,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk_args=(1, 3, 5)),
        dl.PrecisionRecallF1SupportCallback(
            input_key="logits", target_key="targets", num_classes=10
        ),
        dl.AUCCallback(input_key="logits", target_key="targets"),
        # catalyst[ml] required ``pip install catalyst[ml]``
        dl.ConfusionMatrixCallback(input_key="logits", target_key="targets", num_classes=10),
    ],
    logdir=Path("logs") / datetime.now().strftime("%Y%m%d-%H%M%S"),
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
    load_best_on_end=True,
)
# model inference
for prediction in runner.predict_loader(loader=loaders["valid"]):
    assert prediction["logits"].detach().cpu().numpy().shape[-1] == 10

In [ ]:
# Do you forget about logging?

%load_ext tensorboard
%tensorboard --logdir logs